In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

In [35]:
import cv2
import pathlib

In [2]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [3]:
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'

In [30]:
def load_model(model_name):
    base_url = 'http://download.tensorflow.org/models/object_detection/'
    model_file = model_name + '.tar.gz'
    
    model_dir = tf.keras.utils.get_file(
      fname=model_name.replace('/','_'), 
      origin=base_url + model_file,
      untar=True)

    model_dir = os.path.join(pathlib.Path(model_dir),"saved_model")
    model = tf.saved_model.load(str(model_dir))
    return model

In [5]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join(os.getcwd(),'mscoco_label_map.pbtxt')
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [7]:
def run_inference_for_single_image(model, image):
    image = np.asarray(image)
    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis,...]

    # Run inference
    model_fn = model.signatures['serving_default']
    output_dict = model_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(output_dict.pop('num_detections'))
    output_dict = {key:value[0, :num_detections].numpy() for key,value in output_dict.items()}
    output_dict['num_detections'] = num_detections

    # detection_classes should be ints.
    output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
    # Handle models with masks:
    if 'detection_masks' in output_dict:
        # Reframe the the bbox mask to the image size.
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(tf.convert_to_tensor(output_dict['detection_masks']),tf.convert_to_tensor(output_dict['detection_boxes']), image.shape[0], image.shape[1])      
        detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5, tf.uint8)
        output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
    return output_dict

In [33]:
model_name_01 = 'ssd_mobilenet_v1_coco_2017_11_17'
detection_model = load_model(model_name_01)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [36]:
cap = cv2.VideoCapture(0)
try:
    while True:
        ret, image_np = cap.read()
        print(image_np_expanded.shape)
        output_dict = run_inference_for_single_image(detection_model, image_np)
    
        # Visualization of the results of a detection.
        vis_util.visualize_boxes_and_labels_on_image_array(
          image_np,
          output_dict['detection_boxes'],
          output_dict['detection_classes'],
          output_dict['detection_scores'],
          category_index,
          instance_masks=output_dict.get('detection_masks_reframed', None),
          use_normalized_coordinates=True,
          line_thickness=4)
    
        cv2.imshow('Object Detection', cv2.resize(image_np, (800,600)))
    
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            break
except Exception as e:
            print(e)
            cap.release()

(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 3)
(480, 1, 640, 